# Bonne pratiques de programmation

## Convention de nommage (Namming convention)
>Mettre en place une règle de nommage est essentiel pour assurer la lisibilité, la cohérence et la maintenabilité d'un projet, quel que soit son domaine (fichiers, programmation, cloud, réseaux, etc.). Sans une convention bien définie, les noms deviennent rapidement incohérents, difficiles à comprendre et sources d'erreurs.
***
Nous proposons l'utilisation de la convention de nommage recommandé par le Python as travers la documentation PEP8, voici quelques exemples:
- Class: Chaque mot commence par une majuscule tout les autre lettre sont en minuscules ex: DataLoader
- Fonction: Tout en miniscule, un underscore sépare chaque mots ex: load_document
- Constantes: Tout en majuscule, un underscore sépare chaque mots ex: MAX_VALUE
- Variable: Tout en miniscule, un underscore sépare chaque mots ex: montant_total

Vous n'avez pas a vous souciez d'apprendre ces règles par coeur, puisque l'assistant AI intégré peux le faire pour vous:
- Selectionner la cellule ci-dessous
- Activer l'assistant AI en mode Chat (Barre d'outil à droite, button en forme d'étoile).
- Écrivez: Format using PEP8.
- Si vous désirez des explications écrivez: Explain
- Au besoins, apporter les modifications proposées.

_ref: https://peps.python.org/pep-0008/#naming-conventions_

In [0]:
montant_epicerie = 100  # bon
montantVehicule = 300  # Mauvais
montant-taxes = 30  # Mauvais
MontantTotal = 0  # Mauvais

MontantTotal = montant_epicerie + montantVehicule + montant-taxes

display(MontantTotal)

## Refactoring
>Le refactoring de code est une technique couramment utilisée en programmation informatique, et notamment pour le Data Engineering. Elle consiste à restructurer le code informatique sans modifier son comportement externe ou sa fonctionnalité.
***
Dans le cadre d'un expérimentation avec notebook le refactoring va viser deux objectif:
- Pendant l'expériementation:
  - Organiser les cellules de façon à être plus efficient pendant l'experimentation.
- Après l'expérimentation:
  - Organiser le code afin qu'il soit plus lisible et plus facile à maintenir (utilisation de fonctions).

Les étapes du réfactoring sont les suivantes:
1. On code l'experiementation et l'execute sans erreur pour une première fois.
1. On réorganise les cellules afin que les prochaines itérations de l'experiemention soient pous efficientes
1. On réorganise le code:
    1. Créer des fonctions pour alléger et réutiliser le code.
    1. On donne des nom significatifs aux variables et fonctions (rend le code facile a lire et plus compréhensible)
1. On execute l'experimentation à nouveau pour s'assurer que les changements n'ont pas altérer les résultats attendu.
Voir les cellules ci-bas comme exemple complet.

### Réfactoring - Étape 1
On code l'experiementation et l'execute sans erreur pour une première fois.


In [0]:
import numpy as np
import pyspark.pandas as pd

df = pd.read_csv("dbfs:/databricks-datasets/nyctaxi/tripdata/fhv/fhv_tripdata_2015-01.csv.gz")

df["Year"] = df["Pickup_date"].dt.year
df["Month"] = df["Pickup_date"].dt.month
df["Day"] = df["Pickup_date"].dt.day
df["Hour"] = df["Pickup_date"].dt.hour
df = df.drop(["locationID","Pickup_date"], axis=1)

power = 2
sum_of_hours = df["Hour"].sum()
nb_hours = len(df["Hour"])
mse = np.square(sum_of_hours / nb_hours)

print(f"Somme: {sum_of_hours}, Nb heures: {nb_hours}, MSE: {mse}")

### Réfactoring - Étape 2
On réorganise les cellules afin que les prochaines itérations de l'experiemention soient pous efficientes.
- On met les libraries et le chargement du document dans une cellule
- On met la transformations de données dans une autre cellule, cela permet de transformer de différentes façon les données sans avoir à recharger les données à chaque fois.
- On met le calcul dans une cellule à part, ceci permet de modifier le calculs sans avoir à recharger les données et les transformer

In [0]:
import numpy as np
import pyspark.pandas as pd

df = pd.read_csv("dbfs:/databricks-datasets/nyctaxi/tripdata/fhv/fhv_tripdata_2015-01.csv.gz")

In [0]:
df["Year"] = df["Pickup_date"].dt.year
df["Month"] = df["Pickup_date"].dt.month
df["Day"] = df["Pickup_date"].dt.day
df["Hour"] = df["Pickup_date"].dt.hour
df = df.drop(["locationID","Pickup_date"], axis=1)

In [0]:
power = 2
sum_of_hours = df["Hour"].sum()
nb_hours = len(df["Hour"])
mse = np.square(sum_of_hours / nb_hours)

print(mse)

### Réfactoring - Étape 3
On réorganise le code:
- Créer des fonctions pour alléger et réutiliser le code.
- On donne des nom significatifs aux variables et fonctions (rend le code facile a lire et plus compréhensible)

In [0]:
def chargement_donnes(fichier):
    repertoire = "dbfs:/databricks-datasets/nyctaxi/tripdata/fhv/"
    url = repertoire + fichier
    return pd.read_csv(url)

In [0]:
def transformations_donnees(df):
    df["Year"] = df["Pickup_date"].dt.year
    df["Month"] = df["Pickup_date"].dt.month
    df["Day"] = df["Pickup_date"].dt.day
    df["Hour"] = df["Pickup_date"].dt.hour
    return df.drop(["locationID","Pickup_date"], axis=1)

In [0]:
def calculer_mse(df):
    power = 2
    somme_heures = df["Hour"].sum()
    nb_heures = len(df["Hour"])
    mse = np.square(somme_heures / nb_heures)
    return mse

### Réfactoring - Étape 4
On execute l'experimentation à nouveau pour s'assurer que les changements n'ont pas altérer les résultats attendu. 
**Vous pouvez constatez dans la cellule ci-bas a quel point le code est plus lisible et facile a comprendre apres le refactoring**

In [0]:
import numpy as np
import pyspark.pandas as pd

fichier = "fhv_tripdata_2015-01.csv.gz"

df = chargement_donnes(fichier)
df = transformations_donnees(df)
mse = calculer_mse(df)

print(mse)

## Test
>Il existe plusieurs type de tests, nous recommendons, dans le contexte d'une experimentation, d'utiliser le test fonctionel avant de migrer votre code vers votre environnement de production:
***
- Vous devez garantir que les modifications que vous avez apportées n'ont pas:
  - Introduit des erreurs.
  - Des comportements non désirés.

Une fois le code assemblé dans la branche develop, la prochaine étapes serait d'executer à nouveau le notebook et vérifier les résultats.

Une fois le notebook testé, vous pouvez déployer en production avec l'assurance que vos modifications n'ammènrons pas de conséquence négative pour l'organisation.



### Les tests unitaires & les assertions.
> Le test unitaire est une procédure permettant de vérifier le bon fonctionnement d'une partie précise d'un logiciel ou d'une portion d'un programme appelée « unité ». Au niveau de Python l'unité que l'on test est la fonction. Dans le cas d'une experimentation, nous allons faire des tests unitaire pour les raisons suivantes:
- Valider les résultats d'un calcule suite à: 
    - Une modification de code.
    - Un merge des branches features vers la branche develop.
    - Garantir que le résultat du calcul en production sera juste et précis
***
Les étapes sont les suivantes:
- Refactorer le code du calcul dans une fonction.
- Créer une deuxième fonction avec pour recevoir le test.
    - Utiliser des assert pour tester les valeurs en sortie.
    - Les asserts test si une condition est VRAI.
- Executer le test
***
[IMPORTANT] Utilisez les asserts seulement dans la fonction de test et j'amais dans du code qui pourrait allez en production pour les raisons suivantes:
- Les asserts peuvent créer des trous de sécurité
- Les asserts ne sont pas un mécanisme pour gérer les erreurs mais plutot pour les détecter durant la phase de développement.

Voici un exemple ci-dessous

In [0]:
def calculer_regression(x,m,b):
    return (m*x)+b

In [0]:
def test_calculer_regression():
    assert calculer_regression(0,0,0) == 0
    assert calculer_regression(-2,2,3) == 7
    assert calculer_regression(3,2,-3) == 3